# Week 2

## Section 1: Probability and Hashing

** Stochastic Processes **

An ongoing process where the next state might depend on _both the previous states and some random element._

### 1. Frequentist Probability

#### What's the probability (simulated) of rolling a dice and 5 consecutive ones?

In [2]:
import random

def rollDie():
    return random.choice([1,2,3,4,5,6])

def rollN(n):
    result = ''
    for i in range(n):
        result = result + str(rollDie())
    return result

In [4]:
print rollN(10)

4664154415


In [6]:
type(rollN(10)) # the outcome of the rollN Simulation is a string 

str

In [12]:
def getTarget(goal):
    numTries = 0
    numRolls = len(goal)
    while True:
        numTries += 1
        result = rollN(numRolls)
        if result == goal:
            return numTries

def runSim(goal, numTrials): # run sim call getTarget multiple times
    total = 0
    for i in range(numTrials):
        total += getTarget(goal)
    Average_Num_Tries =  total/float(numTrials)
    print 'Probability =', 1.0/Average_Num_Tries

In [13]:
runSim('11111', 100)

Probability = 0.000147901864155


In [19]:
def atLeastOneOne(numRolls, numTrials):
    numSuccess = 0
    for i in range(numTrials):
        rolls = rollN(numRolls)
        if '1' in rolls:
            numSuccess += 1
        fracSuccess = numSuccess/float(numTrials)
    print fracSuccess

In [22]:
atLeastOneOne(10, 1000)

0.859


#### What's the probability (simulated) of rolling a dice and no ones?

In [17]:
(5.0/6)**5

0.401877572016461

** IF you have 100 students in your class today and you promised to give 100 dollar to whomever that is having his/her birthday today. What is the probability of you not paying the money?**

In [18]:
(364.0/365)**100

0.7600670738159009

### 2. Hashing